# Protein Language Model Embeddings with ESM-2 🧬

**Run this notebook on Google Colab:** [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elkins/synth-pdb/blob/main/docs/tutorials/plm_embeddings.ipynb)

---

## What you'll learn

This notebook demonstrates **ESM-2 protein language model embeddings** via `synth-pdb`.

A protein language model (PLM) is a transformer trained on 250 million protein sequences.
After training, its *internal activations* form per-residue vectors that encode:

| What is encoded | Why it's useful |
|---|---|
| **Evolutionary conservation** | Co-varying positions across species |
| **Structural context** | Buried vs. solvent-exposed |
| **Chemical environment** | Charged / polar / hydrophobic neighbourhoods |
| **Functional signals** | Active-site residues |

All from **sequence alone** — no 3D coordinates.

### Notebook outline
1. Setup & Installation
2. Generate synthetic protein structures with `synth-pdb`
3. Embed sequences with ESM-2 → `(L, 320)` float32 arrays
4. **Visualise** per-residue embedding heatmaps
5. **Cluster** proteins in embedding space with UMAP
6. **Similarity heatmap** — polyalanine vs. polyvaline vs. ubiquitin
7. **Linear structural probe** — predict secondary structure from embeddings alone


## 1. Setup & Installation

> **⚠️ Colab users:** The setup cell installs dependencies and **automatically restarts the kernel** once. This is expected — just wait ~10 seconds and **Run All Cells again**.

In [ ]:
# @title Setup & Installation { display-mode: "form" }
import os
import sys
from pathlib import Path

# ── Local development path (ignored on Colab) ────────────────────────────
try:
    repo_root = Path(".").resolve().parent.parent
    if (repo_root / "synth_pdb").exists():
        if str(repo_root) not in sys.path:
            sys.path.insert(0, str(repo_root))
            print(f"📌 Local library: {repo_root}")
except Exception:
    pass

# ── Colab installation ────────────────────────────────────────────────────
if 'google.colab' in str(get_ipython()):
    if not os.path.exists("plm_installed.marker"):
        print("🔧 Installing synth-pdb[plm] and visualisation dependencies ...")
        get_ipython().run_line_magic('pip', 'install -q "synth-pdb[plm]" umap-learn matplotlib seaborn')
        with open("plm_installed.marker", "w") as f:
            f.write("done")
        print("🔄 Installation complete. KERNEL RESTARTING ...")
        print("⚠️  Wait ~10 seconds, then Run All Cells again.")
        os.kill(os.getpid(), 9)
    else:
        print("✅ Dependencies ready.")
else:
    import synth_pdb
    print(f"✅ Local synth-pdb {synth_pdb.__version__}")
    print("   Make sure you have installed: pip install 'synth-pdb[plm]' umap-learn seaborn")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from synth_pdb.plm import ESM2Embedder

# Instantiate the embedder — model loads lazily on first embed() call
embedder = ESM2Embedder()
print(f"ESM2Embedder ready (model: {embedder.model_name})")
print(f"Embedding dim: {embedder.embedding_dim}  (will be 320 for the default t6_8M model)")

## 2. Generate Synthetic Structures with synth-pdb

We use `synth-pdb` to generate three canonical secondary structures so we can compare
how their sequences are encoded differently in PLM space.

| Protein | Sequence | Expected conformation |
|---|---|---|
| Polyalanine | `AAAAAAAAAAAAAAAAAA` (18 residues) | α-helix (Ala strongly prefers helix) |
| Polyvaline | `VVVVVVVVVVVVVVVVVV` (18 residues) | β-strand (Val β-branching disfavours helix) |
| Polyglycine | `GGGGGGGGGGGGGGGGGG` (18 residues) | Coil (Gly has no side chain → most flexible) |
| Ubiquitin N-term | `MQIFVKTLTGKTITLEVEPSDT` (22 residues) | Mixed: helix + strand |

In [ ]:
# Our test sequences — we'll embed these and compare
sequences = {
    "Poly-Ala (helix)": "AAAAAAAAAAAAAAAAAA",
    "Poly-Val (strand)": "VVVVVVVVVVVVVVVVVV",
    "Poly-Gly (coil)": "GGGGGGGGGGGGGGGGGG",
    "Ubiquitin N-term": "MQIFVKTLTGKTITLEVEPSDT",
    "Villin HP35": "LSDEDFKAVFGMTRSAFANLPLWKQQNLKKEKGLF",
    "Trp-cage": "NLYIQWLKDGGPSSGRPPPS",
}

print("Sequences loaded:")
for name, seq in sequences.items():
    print(f"  {name:25s}  {len(seq):3d} residues  {seq[:20]}{'...' if len(seq)>20 else ''}")

## 3. Embed with ESM-2

The first call downloads the model weights (~30 MB) and loads the transformer.
Subsequent calls are fast (~10 ms per protein on CPU).

**Output shape**: `(L, 320)` — one 320-dimensional float32 vector per residue.

In [ ]:
import time

print("Embedding sequences with ESM-2 t6_8M (320-dim) ...")
print("(First call downloads ~30 MB model weights — takes ~5-30 s)\n")

embeddings = {}
for name, seq in sequences.items():
    t0 = time.time()
    emb = embedder.embed(seq)                  # (L, 320) float32
    dt_ms = (time.time() - t0) * 1000
    embeddings[name] = emb
    print(f"  {name:25s}  shape={emb.shape}  dtype={emb.dtype}  {dt_ms:.1f} ms")

print("\n✅ All embeddings computed.")

## 4. Per-Residue Embedding Heatmaps

Each row = one residue, each column = one of the 320 latent dimensions.

**What to look for:**
- A **uniform heatmap** for polyalanine — all residues see identical local contexts
- **More variation** for ubiquitin — each residue has a unique chemical neighbourhood
- Coloured bands = dimensions consistently activated by a particular residue type

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 8))
axes = axes.flatten()

for ax, (name, emb) in zip(axes, embeddings.items()):
    # Normalise each row (residue) to [0,1] for visual comparison
    row_min = emb.min(axis=1, keepdims=True)
    row_max = emb.max(axis=1, keepdims=True)
    normed = (emb - row_min) / (row_max - row_min + 1e-8)

    im = ax.imshow(normed, aspect="auto", cmap="RdBu_r",
                   interpolation="nearest", vmin=0, vmax=1)
    ax.set_title(f"{name}\n({emb.shape[0]} residues × {emb.shape[1]} dims)",
                 fontsize=10, fontweight="bold")
    ax.set_xlabel("ESM-2 embedding dimension", fontsize=8)
    ax.set_ylabel("Residue position", fontsize=8)
    ax.tick_params(labelsize=7)

fig.colorbar(im, ax=axes[-1], label="Activation (row-normalised)", shrink=0.8)
fig.suptitle("ESM-2 Per-Residue Embedding Heatmaps",
             fontsize=14, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()

print("""
Observations:
  • Poly-Ala rows are nearly identical — every residue sees the same context.
  • Ubiquitin rows vary noticeably — each residue has a unique neighbourhood.
  • Vertical stripes = dimensions strongly activated by a residue-type pattern.
""")

## 5. Sequence Similarity in Embedding Space

We collapse each `(L, 320)` matrix to a single `(320,)` vector by **mean pooling**,
then compute **cosine similarity** between all pairs.

$$\text{sim}(A, B) = \frac{\overline{\text{emb}}(A) \cdot \overline{\text{emb}}(B)}{\|\overline{\text{emb}}(A)\| \cdot \|\overline{\text{emb}}(B)\|}$$

**What to expect:**
- Poly-Ala and Poly-Val: **high** similarity (both simple repetitive peptides)
- Poly-Gly: **lower** similarity to others (Gly is biochemically special — smallest residue)
- Ubiquitin / Villin / Trp-cage: moderate similarity (all are real folded proteins)

In [ ]:
names = list(sequences.keys())
n = len(names)

# Compute mean embeddings  →  (N, 320)
mean_embs = np.stack([embeddings[nm].mean(axis=0) for nm in names])

# Normalise rows → unit vectors, then dot product = cosine similarity
norms = np.linalg.norm(mean_embs, axis=1, keepdims=True)
normed = mean_embs / (norms + 1e-8)
sim_matrix = normed @ normed.T   # (N, N)

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(
    sim_matrix,
    xticklabels=names, yticklabels=names,
    annot=True, fmt=".3f",
    cmap="YlOrRd", vmin=0.5, vmax=1.0,
    linewidths=0.5,
    ax=ax,
)
ax.set_title("ESM-2 Sequence Similarity (cosine, mean-pooled)",
             fontsize=13, fontweight="bold", pad=12)
ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha="right", fontsize=9)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=9)
plt.tight_layout()
plt.show()

# Print commentary
for i, a in enumerate(names):
    for j, b in enumerate(names):
        if j > i:
            s = sim_matrix[i, j]
            label = "high" if s > 0.85 else ("moderate" if s > 0.75 else "low")
            print(f"  {a:25s}  ↔  {b:25s}  sim={s:.3f}  ({label})")

## 6. Residue-Level UMAP Clustering

Each residue is a point in 320-dimensional space. We use **UMAP** to project down to 2D.

**Key question**: do residues cluster by amino acid identity, even though the model
was never told what a residue *is* — only which positions are masked?

The answer is yes: ESM-2 implicitly learns amino acid identity through the pattern
of masked predictions. In UMAP space, Ala, Val, Gly etc. should form distinct clouds.

In [ ]:
try:
    import umap

    # Stack all residue embeddings with labels
    all_embs = []
    all_labels = []
    all_source = []

    for name, seq in sequences.items():
        emb = embeddings[name]       # (L, 320)
        for i, (vec, aa) in enumerate(zip(emb, seq)):
            all_embs.append(vec)
            all_labels.append(aa)   # single-letter amino acid code
            all_source.append(name)

    X = np.stack(all_embs)           # (N_residues, 320)
    labels = np.array(all_labels)    # amino acid 1-letter codes

    print(f"Running UMAP on {len(X)} residue embeddings (320-dim → 2D) ...")
    reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=8, min_dist=0.3)
    Z = reducer.fit_transform(X)     # (N_residues, 2)

    # Colour by amino acid identity
    unique_aas = sorted(set(labels))
    cmap = plt.cm.get_cmap("tab20", len(unique_aas))
    color_map = {aa: cmap(i) for i, aa in enumerate(unique_aas)}
    colors = [color_map[aa] for aa in labels]

    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Left: colour by amino acid
    ax = axes[0]
    for aa in unique_aas:
        mask = labels == aa
        ax.scatter(Z[mask, 0], Z[mask, 1], c=[color_map[aa]],
                   label=aa, s=40, alpha=0.75, edgecolors="white", linewidths=0.3)
    ax.legend(title="Amino acid", bbox_to_anchor=(1.0, 1), loc="upper left",
              fontsize=8, title_fontsize=9, ncol=2)
    ax.set_title("UMAP of ESM-2 residue embeddings\nColoured by amino acid identity",
                 fontweight="bold")
    ax.set_xlabel("UMAP-1")
    ax.set_ylabel("UMAP-2")

    # Right: colour by source protein
    ax = axes[1]
    source_labels = np.array(all_source)
    unique_sources = list(sequences.keys())
    scmap = plt.cm.get_cmap("Set2", len(unique_sources))
    for i, src in enumerate(unique_sources):
        mask = source_labels == src
        ax.scatter(Z[mask, 0], Z[mask, 1], c=[scmap(i)],
                   label=src, s=40, alpha=0.75, edgecolors="white", linewidths=0.3)
    ax.legend(title="Protein", bbox_to_anchor=(1.0, 1), loc="upper left",
              fontsize=8, title_fontsize=9)
    ax.set_title("UMAP of ESM-2 residue embeddings\nColoured by source protein",
                 fontweight="bold")
    ax.set_xlabel("UMAP-1")
    ax.set_ylabel("UMAP-2")

    plt.tight_layout()
    plt.show()

    print("""
What to look for:
  • Same amino acid in different proteins → clusters together   (ESM-2 learned residue identity)
  • Points from the same protein → may also cluster             (ESM-2 learned local context)
  • Poly-Ala / Poly-Val / Poly-Gly → tight single-residue dots (all identical context within each)
""")

except ImportError:
    print("umap-learn not installed. Run: pip install umap-learn")
    print("Then re-run this cell.")

## 7. Linear Secondary Structure Probe

Can a **single linear layer** on top of ESM-2 embeddings predict secondary structure?

We test this with **synthetic labelled data** from `synth-pdb`:
- Generate 60 helical residues (`Poly-Ala`) → label 0
- Generate 60 strand residues (`Poly-Val`) → label 1
- Generate 60 coil residues (`Poly-Gly`) → label 2

Then train a logistic regression (which is just a linear layer + softmax) and evaluate accuracy.

> **Educational point**: this test measures whether secondary structure information is
> **linearly accessible** in the embedding space — i.e., can it be extracted with a
> single matrix multiplication? If yes, it's explicitly encoded, not buried in non-linear
> interactions. Literature: ESM-2 achieves ~70-80% accuracy on real protein SS3 datasets
> with a linear probe.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import warnings

# Build a labelled dataset of residue embeddings
# Label 0 = helix (Ala)  |  Label 1 = strand (Val)  |  Label 2 = coil (Gly)
probe_seqs = [
    ("A" * 30, 0, "Helix (Ala)"),
    ("V" * 30, 1, "Strand (Val)"),
    ("G" * 30, 2, "Coil (Gly)"),
    # Additional diversity: mixed sequences
    ("AAAVVVAAA" * 3, None, "Mixed"),   # ground truth is ambiguous, skip
]

X_probe, y_probe = [], []
for seq, label, desc in probe_seqs:
    if label is None:
        continue
    emb = embedder.embed(seq)   # (L, 320)
    for row in emb:
        X_probe.append(row)
        y_probe.append(label)

X_probe = np.array(X_probe)
y_probe = np.array(y_probe)
print(f"Dataset: {X_probe.shape[0]} residues × {X_probe.shape[1]} embedding dims")
print(f"Classes: 0=Helix ({np.sum(y_probe==0)}), 1=Strand ({np.sum(y_probe==1)}), 2=Coil ({np.sum(y_probe==2)})")

# Standardise features (logistic regression converges better)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_probe)

# 5-fold cross-validation with a linear probe
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    clf = LogisticRegression(max_iter=500, C=1.0, random_state=42)
    cv_scores = cross_val_score(clf, X_scaled, y_probe, cv=5, scoring="accuracy")

print(f"\n5-fold CV accuracy: {cv_scores.mean():.1%} ± {cv_scores.std():.1%}")
print(f"Individual folds: {[f'{s:.1%}' for s in cv_scores]}")

# Fit on full data and show confusion matrix
clf.fit(X_scaled, y_probe)
y_pred = clf.predict(X_scaled)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_probe, y_pred)
fig, ax = plt.subplots(figsize=(5, 4))
disp = ConfusionMatrixDisplay(cm, display_labels=["Helix (Ala)", "Strand (Val)", "Coil (Gly)"])
disp.plot(ax=ax, cmap="Blues", colorbar=False)
ax.set_title(f"Linear Probe on ESM-2 Embeddings\nCV accuracy: {cv_scores.mean():.1%}",
             fontweight="bold", pad=10)
plt.tight_layout()
plt.show()

print("""
Interpretation:
  High accuracy → secondary structure is LINEARLY ENCODED in ESM-2 embeddings.
  The model learned to distinguish helix/strand/coil from sequence context alone,
  with no 3D coordinates in the training data.
  This is the same principle that enables ESMFold to predict full 3D structure.
""")

## 8. Embedding a Structure from synth-pdb

We can also pass a **biotite `AtomArray`** directly — `embed_structure()` extracts
the amino acid sequence and calls `embed()` under the hood.

This is the natural integration point with `synth-pdb`'s structure generator.

In [ ]:
from synth_pdb.generator import generate_pdb_content
import biotite.structure.io.pdb as pdb
import biotite.structure as struc
import io

# Generate a synthetic 20-residue alpha-helix
pdb_content = generate_pdb_content(
    sequence_str="ALA-ALA-ALA-ALA-ALA-GLY-LEU-ALA-ALA-ALA-ALA-ALA-SER-ALA-ALA-ALA-ALA-ALA-ALA-ALA",
    ss_type="helix",
    minimize_energy=False,
)

# Parse into a biotite AtomArray
pdb_file = pdb.PDBFile.read(io.StringIO(pdb_content))
structure = pdb_file.get_structure(model=1)
# Get backbone only
backbone = structure[np.isin(structure.atom_name, ["N", "CA", "C", "O"])]

print(f"Generated structure: {len(struc.get_residues(backbone)[0])} residues, {len(backbone)} atoms")

# Embed directly from the AtomArray
struct_emb = embedder.embed_structure(backbone)   # (L, 320)
print(f"PLM embedding shape: {struct_emb.shape}")
print(f"dtype: {struct_emb.dtype}")

# Plot the embedding
fig, ax = plt.subplots(figsize=(12, 3))
im = ax.imshow(struct_emb.T, aspect="auto", cmap="RdBu_r",
               interpolation="nearest")
ax.set_xlabel("Residue position", fontsize=10)
ax.set_ylabel("ESM-2 dimension (320)", fontsize=10)
ax.set_title("synth-pdb → ESM-2 embedding:  20-residue synthetic helix",
             fontsize=11, fontweight="bold")
plt.colorbar(im, ax=ax, label="Activation")
plt.tight_layout()
plt.show()

print("\nThe embedder extracted the sequence from the AtomArray residue names,")
print("passed it through ESM-2, and returned a (L, 320) float32 matrix.")
print("This can be used directly as node features in a GNN.")

## 9. Summary & Next Steps

### What we demonstrated

| Step | Result |
|---|---|
| `embedder.embed(seq)` | `(L, 320)` per-residue float32 matrix |
| `embedder.embed_structure(arr)` | Same, directly from `AtomArray` |
| Heatmaps | Poly-Ala uniform; ubiquitin diverse — context is encoded |
| Similarity matrix | Repetitive peptides cluster; real proteins moderate similarity |
| UMAP | Residues cluster by amino acid identity in 2D |
| Linear probe | High SS accuracy → structural info **linearly accessible** |

### Using a more powerful model

The API is identical for all ESM-2 variants:

```python
# Current: 8M params, 320-dim
embedder = ESM2Embedder()   

# Better: 35M params, 480-dim (~140 MB)
embedder = ESM2Embedder(model_name="facebook/esm2_t12_35M_UR50D")

# Best for research: 650M params, 1280-dim (~2.5 GB, use GPU)
embedder = ESM2Embedder(model_name="facebook/esm2_t33_650M_UR50D", device="cuda")
```

### Downstream integration ideas

```python
# 1. Feed into the synth-pdb GNN quality scorer as node features
plm_feats = embedder.embed_structure(structure)      # (L, 320)
node_features = np.concatenate([geo_feats, plm_feats], axis=-1)

# 2. Pairwise contact prediction (outer product trick)
emb = embedder.embed(sequence)                       # (L, 320)
outer = np.einsum('id,jd->ij', emb, emb)            # (L, L) — feed into CNN

# 3. Retrieval by function similarity
sim = embedder.sequence_similarity(query_seq, candidate_seq)
```

### Reference

Lin, Z. et al. (2023) *Evolutionary-scale prediction of atomic-level protein structure with a language model.* Science 379, 1123–1130.  
https://doi.org/10.1126/science.ade2574